In [1]:
from collections import Counter
import pandas as pd
import sqlite3
from IPython.display import display, Audio, Video, HTML

In [2]:
# export from anki app and unzip it
cnx = sqlite3.connect('miscs/collection.anki21')
df_dict = {}
for table_name in ['cards', 'col', 'graves', 'notes', 'revlog']:
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", cnx)
    df_dict[table_name] = df

In [3]:
words_with_queue_df = df_dict['cards'].join(df_dict['notes'], lsuffix='_cards', rsuffix='_notes')[["queue", "sfld"]]
learned_set = set(word.lower() for word in words_with_queue_df[words_with_queue_df["queue"] != 0]["sfld"])

``` shell
pip3 install youtube-dl
youtube-dl --proxy socks5://127.0.0.1:1086 --write-sub --sub-lang en --skip-download https://www.youtube.com/watch?v=ZNG7g83lI-s --output "miscs/youtube1"
```

In [39]:
# import nltk
# nltk.set_proxy('http://127.0.0.1:1087')
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')

import string
import glob
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 


porter_stemmer  = PorterStemmer()
bad_words = ['-->','</c>'] 
all_words = []
stop_words = set(stopwords.words('english'))

lemma = WordNetLemmatizer()
for filename in glob.glob("miscs/*.vtt"):
    print(filename)
    with open(filename) as oldfile:
        for line in oldfile:
            if not any(bad_word in line for bad_word in bad_words):
                for word in word_tokenize(line):
                    word_n = lemma.lemmatize(word.lower(),'v')
                    word_n = lemma.lemmatize(word_n,'n')
                    if word_n not in learned_set and word_n not in stop_words:
                        if word_n in string.punctuation or len(word_n) < 3 or "'" in word_n or "." in word_n:
                            continue
                        all_words.append(word_n)

miscs/youtube.en.vtt
miscs/youtube1.en.vtt
miscs/youtube2.en.vtt


In [40]:
lemma.lemmatize('derailleurs')

'derailleurs'

In [41]:
all_words_counter = Counter(all_words)

In [42]:
all_words_counter.most_common(30)

[('derailleur', 48),
 ('cog', 36),
 ('adjuster', 29),
 ('outward', 21),
 ('largest', 21),
 ('shifter', 17),
 ('smallest', 17),
 ('lever', 15),
 ('clockwise', 14),
 ('inward', 10),
 ('pedal', 10),
 ('counterclockwise', 10),
 ('derailleurs', 9),
 ('loosen', 8),
 ('chainring', 8),
 ('chainrings', 7),
 ('linkage', 6),
 ('sprocket', 6),
 ('overview', 5),
 ('smaller', 5),
 ('dial', 4),
 ('increment', 4),
 ('clamp', 4),
 ('webvtt', 3),
 ('caption', 3),
 ('calvin', 3),
 ('jones', 3),
 ('pulley', 3),
 ('hex', 3),
 ('wrench', 3)]

In [9]:
import functools
import translators as ts
import eng_to_ipa as ipa

@functools.lru_cache(maxsize=10000, typed=False)
def get_words_trans(word: str, trans_vendor: str = "alibaba"):
    if trans_vendor == "alibaba":
        chn_meaning = ts.alibaba(word, from_language='en', to_language='zh')
    else:
        chn_meaning = ""
    word_ipa = ipa.convert(word)
    audio_url = "https://dict.youdao.com/dictvoice?audio=" + word
    return (chn_meaning, word_ipa, audio_url)

Using China server backend.


In [43]:
results = []
for (word, freq) in all_words_counter.most_common():
    (chn_meaning, word_ipa, audio_url) = get_words_trans(word)
    print(f'{word} : {word_ipa} : {chn_meaning} : {freq}')
    results.append((word, word_ipa, chn_meaning))
    display(HTML(f'<video controls="" name="media" style="height: 80px;"><source src="{audio_url}" type="audio/mpeg"></video>'))

# then import miscs/words.txt to ankis
with open("miscs/words.txt", 'w') as f:
    for result in results:
        f.writelines(f'{result[0]}\t{result[1]}\t{result[2]}\n')
    

derailleur : derailleur* : 变速器 : 48


cog : kɔg : 齿轮 : 36


adjuster : əˈʤəstər : 调节器 : 29


outward : ˈaʊtwərd : 向外 : 21


largest : ˈlɑrʤəst : 最大的 : 21


...

triple : ˈtrɪpəl : 三重 : 1


crankset : crankset* : 曲柄组 : 1


youtube : ˈjuˌtub : youtube : 1


subscribe : səbˈskraɪb : 订阅 : 1


latest : ˈleɪtəst : 最新 : 1


'ˈædəkwɪtli'

In [117]:
type(reuslt)

str